In [49]:
import html2text

path = "/Users/timoblattner/Desktop/CH_BGer_001_1A-1-2000_2000-05-08.html"

h = html2text.HTML2Text()
h.ignore_links = True

with open(path, "r") as file:
    text = h.handle(file.read())
    print(text)



[AZA 3]

1A.1/2000/hzg

1P.1/2000



_           I. OEFFENTLICHRECHTLICHE ABTEILUNG_  

          ********************************** 



_8\. Mai 2000_  



Es wirken mit: Bundesrichter Aemisegger, Präsident

der I. öffentlichrechtlichen Abteilung, Bundesrichter

Aeschlimann, Bundesrichter Favre und Gerichtsschreiberin

Camprubi.



\---------



In Sachen



_August J u l e n_ , des Severin, Chalet Hermitage, Zermatt,  

Beschwerdeführer, vertreten durch Rechtsanwalt Dr. Thomas

Julen, Haus Aurora, Zermatt,



gegen



_Munizipalgemeinde Z e r m a t t_ ,  

_Kantonale Baukommission des Kantons W a l l i s_ ,  

_Staatsrat des Kantons W a l l i s_ ,  

_Kantonsgericht des Kantons W a l l i s_ , Öffentlich-  

rechtliche Abteilung,



betreffend

          Bauabschlag und Abbruchbefehl 

         (rechtliches Gehör, Ausnahmebewilligung), 

hat sich ergeben:



_A.-_  

August Julen führt in Findeln oberhalb von  

Zermatt in der Landwirtschaftszone ein Bergrestaurant.

Unweit davon in

In [51]:
from bs4 import BeautifulSoup
import os
import json


def get_item(name):
    # link
    path = name + ".html"
    
    # file
    file = open(path, "r")

    # read file
    html = BeautifulSoup(file.read())
    
    # the content of the page
    content = html.find("div", {"id": "content"})

    # we want the Erwägung
    erw = content.find('h1', text="Erwägungen")

    data = {}
    data["name"] = name

    # we take the paragraphs
    paragraphs = []
    
    while erw:
        if erw.find('strong') is not None:
            paragraph = {}
            
            # the index of the paragraph
            index = erw.find('strong').extract().text
            paragraph["index"] = index
            
            # we find the links
            links = erw.findChildren("a", {"class": "LexLink"})
            hrefs = []
            for link in links:
                hrefs.append(link["href"][1:])
                link.extract()
            paragraph["links"] = hrefs
            
            # we remove to get text
            '''
            thinsp = erw.findChildren("span", {"class": "thinsp"})
            for t in thinsp:
                t.extract()
            notes = erw.findChildren("span", {"class": "note"})
            for note in notes:
                note.extract()
            pagebk = erw.findChildren("span", {"class": "pagebreak"})
            for bk in pagebk:
                bk.extract()
                '''
            paragraph["content"] = erw.text
            
            print(erw.text)
            
            # check that we have a big enough text
            if len(paragraph["content"]) > 10:
                paragraphs.append(paragraph)
                
    
        # next paragraph
        erw = erw.findNext("p")
    
    data["paragraphs"] = paragraphs
    return data
        
name = "134 I 293"

item = get_item(name)

with open('json_data.json', 'w') as outfile:
    outfile.write(json.dumps(item, indent=4))

 Bei der Prüfung der Verfassungsmässigkeit eines kantonalen Erlasses im Rahmen der abstrakten Normenkontrolle ist nach der Rechtsprechung des Bundesgerichts massgebend, ob der betreffenden Norm nach anerkannten Auslegungsregeln ein Sinn beigemessen werden kann, der sie mit den angerufenen Verfassungsgarantien vereinbar erscheinen lässt. Das Bundesgericht hebt eine kantonale Norm nur auf, wenn sie sich jeder verfassungskonformen (bzw. mit dem höherstufigen Bundesrecht vereinbaren) Auslegung entzieht, nicht jedoch, wenn sie einer solchen in vertretbarer Weise zugänglich bleibt (BGE 133 I 77 E. 2 S. 79, BGE 133 I 286 E. 4.3 S. 295, je mit Hinweisen). Erscheint eine generell-abstrakte Regelung unter normalen Verhältnissen, wie sie der Gesetzgeber voraussetzen durfte, als verfassungsrechtlich zulässig, so vermag die ungewisse Möglichkeit, dass sie sich in besonders gelagerten Einzelfällen als verfassungswidrig auswirken könnte, ein Eingreifen des Verfassungsrichters im Stadium der abstrakte